In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a kist of 2000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs
# Adding latitudes and longitudes to a list
coordinates = list(lats_lngs)
coordinates

[(-13.55855669654349, -18.886694146584347),
 (-7.395948013699666, -38.65598932873894),
 (-75.03318410397095, 70.10446682522107),
 (-7.256416817038939, -62.85745488341098),
 (64.10834302802922, -161.80260047450892),
 (87.56165931046462, -159.80349549865855),
 (58.143095294279874, 81.74297426366212),
 (-60.33339418308982, -113.57973798196225),
 (-58.326968109864204, -88.71017301403393),
 (-46.421951947933316, -152.4550285309712),
 (11.425205108999606, -25.023006064583853),
 (76.6879118156206, 170.0702553920866),
 (86.58123508115222, 96.02272952562629),
 (18.950079020693977, 59.09286587277063),
 (88.74923862492167, 127.08520861499278),
 (47.24707792718962, 55.20128104340452),
 (83.79351737439833, 126.97066242868988),
 (53.26314225642676, -44.42129921877435),
 (68.2760961705813, -79.717151630063),
 (-80.5688777386131, 144.3084960063482),
 (3.4823611110684, 179.70955391053798),
 (66.09328874365391, 175.91066389172533),
 (-27.71590488745558, 118.05886334365255),
 (-65.99758251230192, 137.938

In [3]:
# Get the nearest city using the citipy module
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
# If the city is unique then add it to the cities list
    if city not in cities:
        cities.append(city)

# Check count to confirm sufficient cities
len(cities)

754

In [4]:
# Build URL for Weather API call using my weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Test if URL works
test_city_url = url + "&q=" + "washington"
test_city_weather = requests.get(test_city_url).json()
test_city_weather

{'coord': {'lon': -120.5015, 'lat': 47.5001},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 41.65,
  'feels_like': 33.94,
  'temp_min': 39.99,
  'temp_max': 43,
  'pressure': 952,
  'humidity': 53},
 'visibility': 10000,
 'wind': {'speed': 15.01, 'deg': 315, 'gust': 21},
 'clouds': {'all': 0},
 'dt': 1617637462,
 'sys': {'type': 3,
  'id': 2011179,
  'country': 'US',
  'sunrise': 1617629508,
  'sunset': 1617676617},
 'timezone': -25200,
 'id': 5815135,
 'name': 'Washington',
 'cod': 200}

In [6]:
# Create an empty list to hold the weather data
weather_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the cities list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Get rain and/or snow data if it is raining and/or snowing in the cities
        try:
            city_rain = city_weather["rain"]["3h"]
        except:
            city_rain = 0   
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0

        # Append city weather information into weather_data list
        weather_data.append({"City": city.title(), 
                             "Country": city_country,
                             "Date": city_date,
                             "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,
                             "Current Description": city_weather_description,
                             "Rain inches (last 3 hrs)": city_rain,
                             "Snow inches (last 3 hrs)": city_snow})        

# If an error is experience, skip the city
    except:
        print("City not found. Skipping...")
        pass       
        
# Indicate that Data Retrieval is complete.
print("--------------------------------")
print("Data Retrieval Complete.        ")
print("--------------------------------")

Beginning Data Retrieval     
-----------------------------
Processing record 1 of Set 1 | georgetown
Processing record 2 of Set 1 | mauriti
Processing record 3 of Set 1 | taolanaro
City not found. Skipping...
Processing record 4 of Set 1 | humaita
Processing record 5 of Set 1 | bethel
Processing record 6 of Set 1 | barrow
Processing record 7 of Set 1 | parabel
Processing record 8 of Set 1 | rikitea
Processing record 9 of Set 1 | punta arenas
Processing record 10 of Set 1 | mataura
Processing record 11 of Set 1 | sao filipe
Processing record 12 of Set 1 | pevek
Processing record 13 of Set 1 | khatanga
Processing record 14 of Set 1 | sur
Processing record 15 of Set 1 | tiksi
Processing record 16 of Set 1 | karaton
Processing record 17 of Set 1 | nanortalik
Processing record 18 of Set 1 | clyde river
Processing record 19 of Set 1 | hobart
Processing record 20 of Set 1 | rungata
City not found. Skipping...
Processing record 21 of Set 1 | anadyr
Processing record 22 of Set 1 | geraldton
Pr

Processing record 37 of Set 4 | zhob
Processing record 38 of Set 4 | itarema
Processing record 39 of Set 4 | samusu
City not found. Skipping...
Processing record 40 of Set 4 | sterling
Processing record 41 of Set 4 | torbay
Processing record 42 of Set 4 | kargasok
Processing record 43 of Set 4 | constitucion
Processing record 44 of Set 4 | erzin
Processing record 45 of Set 4 | victoria
Processing record 46 of Set 4 | ler
Processing record 47 of Set 4 | fairbanks
Processing record 48 of Set 4 | yellowknife
Processing record 49 of Set 4 | labutta
City not found. Skipping...
Processing record 50 of Set 4 | yichun
Processing record 1 of Set 5 | selma
Processing record 2 of Set 5 | korla
Processing record 3 of Set 5 | arraial do cabo
Processing record 4 of Set 5 | jijiang
Processing record 5 of Set 5 | mezen
Processing record 6 of Set 5 | bara
Processing record 7 of Set 5 | castro
Processing record 8 of Set 5 | aykhal
Processing record 9 of Set 5 | olutanga
Processing record 10 of Set 5 | l

Processing record 24 of Set 8 | broken hill
Processing record 25 of Set 8 | phalia
Processing record 26 of Set 8 | goma
Processing record 27 of Set 8 | pailon
Processing record 28 of Set 8 | baykit
Processing record 29 of Set 8 | artyom
Processing record 30 of Set 8 | grand baie
Processing record 31 of Set 8 | porto novo
Processing record 32 of Set 8 | la plata
Processing record 33 of Set 8 | thenzawl
Processing record 34 of Set 8 | kizhinga
Processing record 35 of Set 8 | bukama
Processing record 36 of Set 8 | paramithia
City not found. Skipping...
Processing record 37 of Set 8 | nieves
Processing record 38 of Set 8 | marcona
City not found. Skipping...
Processing record 39 of Set 8 | matay
Processing record 40 of Set 8 | puerto madryn
Processing record 41 of Set 8 | karaul
City not found. Skipping...
Processing record 42 of Set 8 | vestmannaeyjar
Processing record 43 of Set 8 | cajamar
Processing record 44 of Set 8 | viedma
Processing record 45 of Set 8 | old shinyanga
Processing rec

Processing record 12 of Set 12 | esso
Processing record 13 of Set 12 | antofagasta
Processing record 14 of Set 12 | lamu
Processing record 15 of Set 12 | verkhnevilyuysk
Processing record 16 of Set 12 | muli
Processing record 17 of Set 12 | requena
Processing record 18 of Set 12 | quatre cocos
Processing record 19 of Set 12 | utiroa
City not found. Skipping...
Processing record 20 of Set 12 | hearst
Processing record 21 of Set 12 | nagorsk
Processing record 22 of Set 12 | corn island
Processing record 23 of Set 12 | molteno
Processing record 24 of Set 12 | nizhneyansk
City not found. Skipping...
Processing record 25 of Set 12 | brae
Processing record 26 of Set 12 | seguela
Processing record 27 of Set 12 | sambava
Processing record 28 of Set 12 | santa rosalia
Processing record 29 of Set 12 | angra
City not found. Skipping...
Processing record 30 of Set 12 | shimoda
Processing record 31 of Set 12 | maceio
Processing record 32 of Set 12 | verkhnyaya sinyachikha
Processing record 33 of Se

Processing record 45 of Set 15 | bull savanna
Processing record 46 of Set 15 | sujawal
City not found. Skipping...
Processing record 47 of Set 15 | dillon
Processing record 48 of Set 15 | namatanai
Processing record 49 of Set 15 | torrington
Processing record 50 of Set 15 | kijang
Processing record 1 of Set 16 | port-cartier
Processing record 2 of Set 16 | guasdualito
Processing record 3 of Set 16 | barbastro
Processing record 4 of Set 16 | paamiut
--------------------------------
Data Retrieval Complete.        
--------------------------------


In [7]:
# Convert the list to a DataFrame
weather_data_df = pd.DataFrame(weather_data)
weather_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Georgetown,MY,2021-04-05 15:39:47,5.4112,100.3354,84.20,78,40,4.61,light rain,0.0,0
1,Mauriti,BR,2021-04-05 15:44:23,-7.3892,-38.7744,94.19,30,73,7.43,broken clouds,0.0,0
2,Humaita,BR,2021-04-05 15:44:23,-7.5061,-63.0208,92.91,47,77,2.59,broken clouds,0.0,0
3,Bethel,US,2021-04-05 15:44:24,41.3712,-73.4140,55.40,33,1,16.11,clear sky,0.0,0
4,Barrow,US,2021-04-05 15:44:24,71.2906,-156.7887,-7.60,71,40,4.61,scattered clouds,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
683,Kijang,KR,2021-04-05 15:48:08,35.2442,129.2139,48.20,57,0,5.75,clear sky,0.0,0
684,Port-Cartier,CA,2021-04-05 15:48:08,50.0334,-66.8654,41.00,100,90,10.36,mist,0.0,0
685,Guasdualito,VE,2021-04-05 15:48:09,7.2424,-70.7324,80.60,78,94,4.61,moderate rain,0.0,0
686,Barbastro,ES,2021-04-05 15:48:09,42.0356,0.1269,68.00,30,0,13.80,clear sky,0.0,0


In [11]:
# Export to CSV

weather_data_df.to_csv("Weather_Database/WeatherPy_Database.csv")